<a href="https://colab.research.google.com/github/vamshap/PySpark-Challenges/blob/main/City.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("CityPopulation").getOrCreate()

# Define the data as a list of tuples
data = [
    ('haryana', 'ambala', 100),
    ('haryana', 'panipat', 200),
    ('haryana', 'gurgaon', 300),
    ('punjab', 'amritsar', 150),
    ('punjab', 'ludhiana', 400),
    ('punjab', 'jalandhar', 250),
    ('maharashtra', 'mumbai', 1000),
    ('maharashtra', 'pune', 600),
    ('maharashtra', 'nagpur', 300),
    ('karnataka', 'bangalore', 900),
    ('karnataka', 'mysore', 400),
    ('karnataka', 'mangalore', 200),
]

# Define schema
columns = ["state", "city", "population"]

# Create DataFrame
city_population_df = spark.createDataFrame(data, schema=columns)

# Show the DataFrame
city_population_df.show()

+-----------+---------+----------+
|      state|     city|population|
+-----------+---------+----------+
|    haryana|   ambala|       100|
|    haryana|  panipat|       200|
|    haryana|  gurgaon|       300|
|     punjab| amritsar|       150|
|     punjab| ludhiana|       400|
|     punjab|jalandhar|       250|
|maharashtra|   mumbai|      1000|
|maharashtra|     pune|       600|
|maharashtra|   nagpur|       300|
|  karnataka|bangalore|       900|
|  karnataka|   mysore|       400|
|  karnataka|mangalore|       200|
+-----------+---------+----------+



In [ ]:
from pyspark.sql.functions import max,min,aggregate as agg,col


city_df = city_population_df.groupBy("state").agg(max("population").alias("more"),min("population").alias("less"))

city_df.show()
city_population_df.show()

#dd = city_df.join(city_population_df,(city_population_df["state"] == city_df["state"]) & (city_population_df["population"] == city_df["less"]) ,how = "inner").select(city_df["state"],city_df["more"],city_df["less"],city_population_df["city"].alias("less_City"))
dd = city_df.alias("city").join(city_population_df.alias("population"),(col("population.state") == col("city.state")) & (col("population.population") == col("city.less")) ,how = "inner").select(col("city.state"),col("city.more"),col("city.less"),col("population.city").alias("less_City"))


dd.show()

#lessCity_df = dd.select("state","less","more",dd["city"].alias("lessCity"))

result_df = dd.alias("less_df").join(city_population_df.alias("more_df"),(col("less_df.state") == col("more_df.state")) & (col("more_df.population") == col("less_df.more")) ,how = "inner").select(col("less_df.state"),col("less_df.less"),col("less_df.more"),col("more_df.city").alias("more_city"),col("less_df.less_city"))


#result_df = lesCity_df.join(city_population_df,(city_population_df["state"] == city_df["state"]) & (city_population_df["population"] == city_df["more"]) ,how = "inner").select(dd["state"],dd["less_city"],dd["less"],dd["more"],city_population_df["city"])

result_df.show()





+-----------+----+----+
|      state|more|less|
+-----------+----+----+
|    haryana| 300| 100|
|     punjab| 400| 150|
|maharashtra|1000| 300|
|  karnataka| 900| 200|
+-----------+----+----+

+-----------+---------+----------+
|      state|     city|population|
+-----------+---------+----------+
|    haryana|   ambala|       100|
|    haryana|  panipat|       200|
|    haryana|  gurgaon|       300|
|     punjab| amritsar|       150|
|     punjab| ludhiana|       400|
|     punjab|jalandhar|       250|
|maharashtra|   mumbai|      1000|
|maharashtra|     pune|       600|
|maharashtra|   nagpur|       300|
|  karnataka|bangalore|       900|
|  karnataka|   mysore|       400|
|  karnataka|mangalore|       200|
+-----------+---------+----------+

+-----------+----+----+---------+
|      state|more|less|less_City|
+-----------+----+----+---------+
|    haryana| 300| 100|   ambala|
|     punjab| 400| 150| amritsar|
|maharashtra|1000| 300|   nagpur|
|  karnataka| 900| 200|mangalore|
+-------